# We will be removing the keys from the json objects, as they might impact the score.

In [ ]:
import pandas as pd

In [ ]:
# !pip install openpyxl

In [ ]:
NUM_EPOCHS = 10

In [ ]:
results = pd.read_excel(f"mistral_finetuned_output_{NUM_EPOCHS}epochs.xlsx")

In [ ]:
results.head(3)

In [ ]:
print(results.columns)

Prompts: input <br>
Response: Test cases of reference dataset <br>
Output: Test cases by fine tuned LLAMA 3.1 8B model

In [ ]:
print(results.loc[10, "Response"])

## Remove \n and \t

In [ ]:
def remove_whitespace(value):
	return value.strip().replace("\n", " ").replace("\t", " ")

In [ ]:
for type in ["Prompts", "Response", "Output"]:
	results[type] = results[type].apply(lambda x: remove_whitespace(x))

In [ ]:
results.head(3)

In [ ]:
results.shape

# Remove fixed keys

In [ ]:
results_fixed = {} # remove all keys
results_fixed["Response"] = []
results_fixed["Output"] = []
error_indices = []

In [ ]:
print(results.loc[10, "Output"])

In [ ]:
import json
for idx in range(0, 200):
	both_ok = True # check if both are json parseable
	for type in ["Response", "Output"]:
		tcs = results.loc[idx, type]
		try:
			tcs = json.loads(tcs)
		except:
			both_ok = False
			print(type)
			print(idx)
			print(tcs)
	
	if both_ok:
		results_fixed["Response"].append(json.loads(results.loc[idx, "Response"]))
		results_fixed["Output"].append(json.loads(results.loc[idx, "Output"]))
	else:
		error_indices.append(idx)

In [ ]:
print(len(results_fixed["Response"]))
print(len(results_fixed["Output"]))

In [ ]:
error_indices

In [ ]:
def remove_keys(d):
    """
    remove all first level keys from d, convert it to a string
    """
    return str(list(d.values()))[1:-1]

In [ ]:
for type in ["Response", "Output"]:
    for idx, tcs in enumerate(results_fixed[type]):
        new_tcs = []
        if not isinstance(tcs["testcases"], list):
            tcs["testcases"] = [tcs["testcases"]]
        for tc in tcs["testcases"]:
            new_tcs.append(remove_keys(tc))
        # break
        results_fixed[type][idx] = "; ".join(new_tcs)

## For the errored indices, use string replacement

In [ ]:
common_keys = ['"testcases":', '"name":', '"description":', '"input":', '"expected":']

In [ ]:
def remove_vals(s, vals):
    for val in vals:
        s = s.replace(val, "")
    return s

In [ ]:
for idx in error_indices:
	results_fixed["Response"].append(remove_vals(results.loc[idx, "Response"], common_keys))
	results_fixed["Output"].append(remove_vals(results.loc[idx, "Output"], common_keys))

In [ ]:
print(len(results_fixed["Response"]))
print(len(results_fixed["Output"]))

In [ ]:
print(results_fixed["Response"][63])
print(results_fixed["Output"][63])

In [ ]:
print(results_fixed["Response"][-1])
print(results_fixed["Output"][-1])

In [ ]:
# !pip install evaluate
# !pip install absl-py
# !pip install nltk
# !pip install rouge-score
# !pip install transformers
# !pip install bert-score
# !pip install --upgrade huggingface_hub
# !pip install accelerate==0.31.0

In [ ]:
scores = {}

In [ ]:
import evaluate

rouge = evaluate.load("rouge")
	
scores["rouge"] = rouge.compute(predictions=results_fixed["Output"], references=results_fixed["Response"])

print(f"Finetuned scores: {scores["rouge"]}")

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
	
scores["blue"] = bleu.compute(predictions=results_fixed["Output"], references=results_fixed["Response"])

print(f"Finetuned scores: {scores["blue"]}")

In [ ]:
import evaluate

bert = evaluate.load("bertscore")
	
scores["bert"] = bert.compute(predictions=results_fixed["Output"], references=results_fixed["Response"], lang="en")

print(f"Finetuned scores: {scores["bert"]}")

In [ ]:
scores["bert"].keys()

In [ ]:
import numpy as np

print(np.mean(scores["bert"]["precision"]))
print(np.mean(scores["bert"]["recall"]))
print(np.mean(scores["bert"]["f1"]))

In [ ]:
scores["bert"]["precision"] = np.mean(scores["bert"]["precision"])
scores["bert"]["recall"] = np.mean(scores["bert"]["recall"])
scores["bert"]["f1"] = np.mean(scores["bert"]["f1"])

In [ ]:
import json
with open(f"mistral_scores_final_fixed_finetuned_{NUM_EPOCHS}epochs.json", "w") as fp:
	json.dump(scores, fp)